In [1]:
import pandas as pd

In [4]:
file = "C:/Users/tomek/Desktop/CME_20181029.csv"

In [12]:
df = pd.read_csv(file,
                 error_bad_lines=False,
                 header=None,
                 parse_dates=[1],
                 names=['symbol', 'date', 'open', 'high', 'low', 'last', 'change', 'settle', 'volume', 'open_interest', 'x', 'y'])

In [13]:
df.count()

symbol           15121598
date             15121598
open              5725179
high              5827020
low               5827200
last               706823
change0           8626097
settle           15100578
volume           14976931
open_interest    14998092
x                    2066
y                   15410
dtype: int64

In [16]:
df.describe

<bound method NDFrame.describe of            symbol       date  open  high  low  last  change0     settle  \
0         0DF2015 2014-02-18   NaN   NaN  NaN   NaN  4.59600  578.57100   
1         0DF2015 2014-02-19   NaN   NaN  NaN   NaN  0.86200  577.70900   
2         0DF2015 2014-02-20   NaN   NaN  NaN   NaN  0.92500  578.63400   
3         0DF2015 2014-02-21   NaN   NaN  NaN   NaN  1.76800  576.86600   
4         0DF2015 2014-02-24   NaN   NaN  NaN   NaN  3.26100  580.12700   
5         0DF2015 2014-02-25   NaN   NaN  NaN   NaN  6.83600  573.29100   
6         0DF2015 2014-02-26   NaN   NaN  NaN   NaN  1.30600  574.59700   
7         0DF2015 2014-02-27   NaN   NaN  NaN   NaN  1.17300  573.42400   
8         0DF2015 2014-02-28   NaN   NaN  NaN   NaN  1.69200  575.11600   
9         0DF2015 2014-03-03   NaN   NaN  NaN   NaN  7.42300  582.53900   
10        0DF2015 2014-03-04   NaN   NaN  NaN   NaN  8.95700  573.58200   
11        0DF2015 2014-03-05   NaN   NaN  NaN   NaN  6.83900  566.

In [14]:
df.head()

,symbol,date,open,high,low,last,change0,settle,volume,open_interest,x,y
0,0DF2015,2014-02-18,NaN,NaN,NaN,NaN,4.596,578.571,0.0,10.0,NaN,NaN
1,0DF2015,2014-02-19,NaN,NaN,NaN,NaN,0.862,577.709,0.0,10.0,NaN,NaN
2,0DF2015,2014-02-20,NaN,NaN,NaN,NaN,0.925,578.634,0.0,10.0,NaN,NaN
3,0DF2015,2014-02-21,NaN,NaN,NaN,NaN,1.768,576.866,3.0,10.0,NaN,NaN
4,0DF2015,2014-02-24,NaN,NaN,NaN,NaN,3.261,580.127,0.0,13.0,NaN,NaN


In [22]:
df[df['symbol'] == 'ESZ2018'][df['date'] > '2018'].tail()

C:\Users\tomek\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,symbol,date,open,high,low,last,change0,settle,volume,open_interest,x,y
4726015,ESZ2018,2018-10-23,2755.75,2757.50,2692.25,2746.00,10.25,2746.25,2780303.0,2744048.0,NaN,NaN
4726016,ESZ2018,2018-10-24,2745.25,2750.00,2652.25,2674.75,82.00,2664.25,2783685.0,2762871.0,NaN,NaN
4726017,ESZ2018,2018-10-25,2673.75,2723.75,2658.75,2681.50,24.00,2688.25,2479997.0,2780617.0,NaN,NaN
4726018,ESZ2018,2018-10-26,2678.00,2692.75,2627.25,2662.25,18.75,2669.50,3410150.0,2777378.0,NaN,NaN
4726019,ESZ2018,2018-10-29,2667.00,2707.00,2603.00,2639.50,26.00,2643.50,2720779.0,2810469.0,NaN,NaN


In [29]:
df[~df['x'].isnull()].head()

,symbol,date,open,high,low,last,change0,settle,volume,open_interest,x,y


In [23]:
df['date'].min()

Timestamp('1959-07-01 00:00:00')

In [24]:
df['date'].max()

Timestamp('2018-10-29 00:00:00')

In [27]:
# drop option codes which are mistakenly included in Quandl file
df.drop(df[df['symbol'].str.len() > 8].index, inplace=True)
# drop various indexes included in Quandl file
df.drop(df[df['symbol'].str.contains('INDEX', regex=False) == True].index, inplace=True)

In [30]:
del df['x']
del df['y']

In [28]:
df[df['symbol'].str.contains('INDEX', regex=False) == True]

,symbol,date,open,high,low,last,change0,settle,volume,open_interest,x,y


In [31]:
df.head()

,symbol,date,open,high,low,last,change0,settle,volume,open_interest
0,0DF2015,2014-02-18,NaN,NaN,NaN,NaN,4.596,578.571,0.0,10.0
1,0DF2015,2014-02-19,NaN,NaN,NaN,NaN,0.862,577.709,0.0,10.0
2,0DF2015,2014-02-20,NaN,NaN,NaN,NaN,0.925,578.634,0.0,10.0
3,0DF2015,2014-02-21,NaN,NaN,NaN,NaN,1.768,576.866,3.0,10.0
4,0DF2015,2014-02-24,NaN,NaN,NaN,NaN,3.261,580.127,0.0,13.0
